# Walkthrough
https://docs.wandb.ai/guides/sweeps/walkthrough

This page shows how to define, initialize, and run a sweep. There are four main steps:
1. [Set up your training code](https://docs.wandb.ai/guides/sweeps/walkthrough#set-up-your-training-code)
1. [Define the search space with a sweep configuration](https://docs.wandb.ai/guides/sweeps/walkthrough#define-the-search-space-with-a-sweep-configuration)
1. [Initialize the sweep](https://docs.wandb.ai/guides/sweeps/walkthrough#initialize-the-sweep)
1. [Start the sweep agent](https://docs.wandb.ai/guides/sweeps/walkthrough#start-the-sweep)

In [1]:
# Import the W&B Python Library and log into W&B
import wandb
wandb.login()

# define training function (args: config, and optionally more)
def training(config, sweep_index):
    score = len(config.lora_bias)**3 + config.lora_rank
    print(sweep_index)
    return score, sweep_index+1

sweep_index = 1
project = "ner_fewnerd_lora"

# define the function (here: main) which the sweep agent will call (args: none)
def main():
    global sweep_index
    wandb.init()
    score, sweep_index = training(wandb.config, sweep_index)
    wandb.log({"score": score})

# define the hyperparameter space
sweep_configuration = {
    "method": "grid",
    "metric": {"goal": "minimize", "name": "score"},
    "parameters": {
        "lora_bias": {"values": ["none", "lora_only"]}, #  3 values ["none", "lora_only", "all"]
        "lora_rank": {"values": [1, 2]},                # 10 values [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
        "model_version": {"values": ["base", "large"]}, #  2 values ["base", "large"]
        # separate 3 sweeps by manually changing the 3 "lora_dropout" values [0, 0.25, 0.5]
        "split": {"values": ["dev", "1"]}               #  4 values ["dev", "1", "10", "100"]
        # the bottom parameter key (here: "split") is sweeped before above parameters
    },
}

# declare the sweep (config and project)
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project)

count = 1 # initial value before computation of "count"
for key in sweep_configuration["parameters"].keys():
    count *= len(sweep_configuration["parameters"][key]["values"])
#
print(f"number of sweeps: {count}")
wandb.agent(sweep_id, function=main, count=count)

Create sweep with ID: wp16bmys
Sweep URL: https://wandb.ai/mdroth/ner_fewnerd_lora/sweeps/wp16bmys
number of sweeps: 16


wandb: Agent Starting Run: hakczpez with config:
wandb: 	lora_bias: none
wandb: 	lora_rank: 1
wandb: 	model_version: base
wandb: 	split: dev


1


wandb: Agent Starting Run: a59wa9e6 with config:
wandb: 	lora_bias: none
wandb: 	lora_rank: 1
wandb: 	model_version: base
wandb: 	split: 1


2


...

14


wandb: Agent Starting Run: hacq43o4 with config:
wandb: 	lora_bias: lora_only
wandb: 	lora_rank: 2
wandb: 	model_version: large
wandb: 	split: dev


15


wandb: Agent Starting Run: 1yc825id with config:
wandb: 	lora_bias: lora_only
wandb: 	lora_rank: 2
wandb: 	model_version: large
wandb: 	split: 1


16


The following sections break down and explains each step in the code sample.

## Set up your training code
Define a training function that takes in hyperparameter values from `wandb.config` and uses them to train a model and return metrics.

Optionally provide the name of the project where you want the output of the W&B Run to be stored (project parameter in `wandb.init`). If the project is not specified, the run is put in an "Uncategorized" project.

> <font color="darkgreen">💡 TIP<br>Both the sweep and the run must be in the same project. Therefore, the name you provide when you initialize W&B must match the name of the project you provide when you initialize a sweep.</font>


```python
# 1: Define objective/training function
def objective(config):
    score = config.x**3 + config.y
    return score

def main():
    wandb.init(project="my-first-sweep")
    score = objective(wandb.config)
    wandb.log({"score": score})
```

## Define the search space with a sweep configuration
Within a dictionary, specify what hyperparameters you want to sweep over and. For more information about configuration options, see Define sweep configuration.

The proceeding example demonstrates a sweep configuration that uses a random search (`"method":"random"`). The sweep will randomly select a random set of values listed in the configuration for the batch size, epoch, and the learning rate.

Throughout the sweeps, W&B will maximize the metric specified in the metric key (`metric`). In the following example, W&B will maximize (`"goal": "maximize"`) the validation accuracy (`"val_acc"`).

```python
# 2: Define the search space
sweep_configuration = {
    "method": "random",
    "metric": {"goal": "minimize", "name": "score"},
    "parameters": {
        "x": {"max": 0.1, "min": 0.01},
        "y": {"values": [1, 3, 7]},
    },
}
```
## Initialize the sweep
W&B uses a *Sweep Controller* to manage sweeps on the cloud (standard), locally (local) across one or more machines. For more information about Sweep Controllers, see [Search and stop algorithms locally](https://docs.wandb.ai/guides/sweeps/local-controller).

A sweep identification number is returned when you initialize a sweep:

```python
sweep_id = wandb.sweep(sweep=sweep_configuration, project="my-first-sweep")
```
For more information about initializing sweeps, see [Initialize sweeps](https://docs.wandb.ai/guides/sweeps/initialize-sweeps).

## Start the Sweep
Use the `wandb.agent` API call to start a sweep.
```python
wandb.agent(sweep_id, function=main, count=10)
```

## Visualize results (optional)
Open your project to see your live results in the W&B App dashboard. With just a few clicks, construct rich, interactive charts like [parallel coordinates plots](https://docs.wandb.ai/guides/app/features/panels/parallel-coordinates), [parameter importance analyzes](https://docs.wandb.ai/guides/app/features/panels/parameter-importance), and [more](https://docs.wandb.ai/guides/app/features/panels).

<img style="text-align: center" width="90%" src="https://docs.wandb.ai/assets/images/quickstart_dashboard_example-ef2f7996d83febe92abee2d092dc0c12.png" />

For more information about how to visualize results, see [Visualize sweep results](https://docs.wandb.ai/guides/sweeps/visualize-sweep-results). For an example dashboard, see this sample [Sweeps Project](https://wandb.ai/anmolmann/pytorch-cnn-fashion/sweeps/pmqye6u3).

## Stop the agent (optional)
From the terminal, hit `Ctrl+c` to stop the run that the Sweep agent is currently running. To kill the agent, hit `Ctrl+c` again after the run is stopped.

$\checkmark$